In [1]:
import cv2
import numpy as np

def chat_gpt_cartoonize(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply median blur to reduce noise
    gray = cv2.medianBlur(gray, 5)

    # Detect edges using Laplacian operator
    edges = cv2.Laplacian(gray, cv2.CV_8U, ksize=5)

    # Apply adaptive thresholding to emphasize edges
    threshold = cv2.adaptiveThreshold(edges, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=9, C=2)

    # Apply bilateral filter to smooth the image while keeping edges sharp
    filtered = cv2.bilateralFilter(image, 9, 75, 75)

    # Combine the thresholded edges with the filtered image
    cartoon = cv2.bitwise_and(filtered, filtered, mask=threshold)

    return cartoon

def made_cartoonize(image):
    # 이미지를 그레이 스케일로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 노이즈를 줄이기 위해 median blur 적용
    gray = cv2.medianBlur(gray, 5)

    # 에지를 강조하기 위해 adaptive thresholding 적용
    threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=7, C=5)

    # Bilateral filter를 이용하여 에지는 유지하면서 이미지를 부드럽게 만듦
    filtered = cv2.bilateralFilter(image, 9, 75, 75)

    # 색상 양자화
    quantizaton = color_quantization(filtered,50)
    
    # thresholded edges와 filtered image를 합성
    cartoon = cv2.bitwise_and(quantizaton, quantizaton, mask=threshold)
    # cartoon = cv2.stylization(image, sigma_s=150, sigma_r=0.25)

    return cartoon

def color_quantization(img, k):
    
  data = np.float32(img).reshape((-1, 3))

  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    
  ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

  center = np.uint8(center)
    
  result = center[label.flatten()]

  result = result.reshape(img.shape)
    
  return result

# 이미지 불러오기
image = cv2.imread('data/lena.tif')

# stylization함수
stylization_cartoonize = cv2.stylization(image, sigma_s=150, sigma_r=0.25)

# 원본 이미지와 만화 스타일 이미지를 옆으로 나란히 보여줌
cv2.imshow('Original', image)
cv2.imshow('chat_gpt_cartoonize', chat_gpt_cartoonize(image))
cv2.imshow('made_cartoonize', made_cartoonize(image))
cv2.imshow('stylization_cartoonize', stylization_cartoonize)

cv2.waitKey(0)
cv2.destroyAllWindows()